# Judgement Panda Exfil Activity
Detects Russian group activity as described in Global Threat Report 2019 by Crowdstrike

## Rule Content
```
- title: Judgement Panda Exfil Activity
  id: b83f5166-9237-4b5e-9cd4-7b5d52f4d8ee
  description: Detects Russian group activity as described in Global Threat Report
    2019 by Crowdstrike
  references:
  - https://www.crowdstrike.com/resources/reports/2019-crowdstrike-global-threat-report/
  author: Florian Roth
  date: 2019/02/21
  tags:
  - attack.credential_access
  - attack.t1081
  - attack.t1003
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      Image: '*\xcopy.exe'
      CommandLine: '* /S /E /C /Q /H \\*'
    selection2:
      Image: '*\adexplorer.exe'
      CommandLine: '* -snapshot "" c:\users\\*'
    condition: selection1 or selection2
  falsepositives:
  - unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\xcopy.exe AND process_command_line.keyword:*\ \/S\ \/E\ \/C\ \/Q\ \/H\ \\*) OR (process_path.keyword:*\\adexplorer.exe AND process_command_line.keyword:*\ \-snapshot\ \"\"\ c\:\\users\\*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()